In [43]:
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [44]:
request = ['академическая', 37.495123, 55.840042, 45, 'gasSilicateBlock', 2, 298892882, 43.5, 33, 11266065.0]

In [45]:
df = pd.read_csv('0105PARS55k.csv')
df = df.drop(columns = ['Unnamed: 0'])

scaler = StandardScaler()
df[['lon_norm', 'lat_norm', 'Area_norm', 'floor_norm']] = scaler.fit_transform(df[['lon', 'lat', 'Area', 'floor']])

In [46]:
columns = ['Metro station', 'lon',
       'lat', 'Number of floors', 'type', 'Number of rooms', 'id',
       'Area', 'floor', 'Price']
request_df = pd.DataFrame([request], columns=columns)

fets = scaler.transform(request_df[['lon', 'lat', 'Area', 'floor']]).tolist()[0]

sames = df[(df['type'] == request[4]) & (df['Number of rooms'] == request[5])]

In [47]:
w_lon = 2
w_lat = 2
w_area = 1
w_floor = 0.2

sames.loc[:, 'dist2'] = w_lon * (sames['lon_norm'] - fets[0]) ** 2 + w_lat * (sames['lat_norm'] - fets[1]) ** 2 + w_area * (sames['Area_norm'] - fets[2]) ** 2 + w_floor * (sames['floor_norm'] - fets[3]) ** 2
sames = sames.sort_values(by = 'dist2') 
sames.head(5)

/var/folders/mz/c85xwc6170v72j1f_tsm0bkh0000gn/T/ipykernel_31014/814681954.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sames.loc[:, 'dist2'] = w_lon * (sames['lon_norm'] - fets[0]) ** 2 + w_lat * (sames['lat_norm'] - fets[1]) ** 2 + w_area * (sames['Area_norm'] - fets[2]) ** 2 + w_floor * (sames['floor_norm'] - fets[3]) ** 2


,Metro station,lon,lat,Number of floors,type,Number of rooms,id,Area,floor,Price,date,lon_norm,lat_norm,Area_norm,floor_norm,dist2
43240,рабочийпосёлок,37.413421,55.730390,16,gasSilicateBlock,2.0,300299862,53.1,16,15800000.0,2024-05-01,-0.065007,-0.176485,-0.027539,0.827704,0.921891
51533,владыкино,37.586607,55.849221,16,gasSilicateBlock,2.0,299739639,38.6,14,8990000.0,2024-05-01,-0.013543,-0.019838,-0.071506,0.581195,1.098829
38625,депо,37.723520,55.676585,16,gasSilicateBlock,2.0,298228454,53.0,14,15150000.0,2024-05-01,0.027143,-0.247412,-0.027842,0.581195,1.199738
7871,новогиреево,37.806299,55.755834,14,gasSilicateBlock,2.0,300810945,47.0,13,13200000.0,2024-04-30,0.051741,-0.142944,-0.046035,0.457941,1.257191
43187,молодёжная,37.408948,55.747034,12,gasSilicateBlock,2.0,300551181,37.4,12,13900000.0,2024-05-01,-0.066336,-0.154544,-0.075145,0.334686,1.371622


In [48]:
w_lon = 1.8
w_lat = 1.8
w_area = 0.8
w_floor = 0.2

sames.loc[:, 'dist2'] = w_lon * (sames['lon_norm'] - fets[0]) ** 2 + w_lat * (sames['lat_norm'] - fets[1]) ** 2 + w_area * (sames['Area_norm'] - fets[2]) ** 2 + w_floor * (sames['floor_norm'] - fets[3]) ** 2
sames = sames.sort_values(by = 'dist2') 
sames.head(5)

,Metro station,lon,lat,Number of floors,type,Number of rooms,id,Area,floor,Price,date,lon_norm,lat_norm,Area_norm,floor_norm,dist2
43240,рабочийпосёлок,37.413421,55.730390,16,gasSilicateBlock,2.0,300299862,53.1,16,15800000.0,2024-05-01,-0.065007,-0.176485,-0.027539,0.827704,0.917425
51533,владыкино,37.586607,55.849221,16,gasSilicateBlock,2.0,299739639,38.6,14,8990000.0,2024-05-01,-0.013543,-0.019838,-0.071506,0.581195,1.098608
38625,депо,37.723520,55.676585,16,gasSilicateBlock,2.0,298228454,53.0,14,15150000.0,2024-05-01,0.027143,-0.247412,-0.027842,0.581195,1.189365
7871,новогиреево,37.806299,55.755834,14,gasSilicateBlock,2.0,300810945,47.0,13,13200000.0,2024-04-30,0.051741,-0.142944,-0.046035,0.457941,1.252993
43187,молодёжная,37.408948,55.747034,12,gasSilicateBlock,2.0,300551181,37.4,12,13900000.0,2024-05-01,-0.066336,-0.154544,-0.075145,0.334686,1.368416


In [49]:
print('The best match: ', f'\n https://www.cian.ru/sale/flat/{sames.iloc[0, 6]}/')
print('The second: ', f'\n https://www.cian.ru/sale/flat/{sames.iloc[1, 6]}/')
print('The third: ', f'\n https://www.cian.ru/sale/flat/{sames.iloc[2, 6]}/')

The best match:  
 https://www.cian.ru/sale/flat/300299862/
The second:  
 https://www.cian.ru/sale/flat/299739639/
The third:  
 https://www.cian.ru/sale/flat/298228454/


In [50]:
request = ['Авиамоторная', 37.495123, 55.840042, 45, 'gasSilicateBlock', 2, 298892882, 43.5, 33, 11266065.0]

In [51]:
metrostation = []
mtypes = []
for i in df['Metro station']: metrostation.append(i) if i not in metrostation else i
for i in df['type']: mtypes.append(i) if i not in mtypes else i

In [52]:
metrostation

['авиамоторная',
 'автозаводская',
 'академическая',
 'александровскийсад',
 'кокошкино',
 'алексеевская',
 'алтуфьево',
 'аннино',
 'южная',
 'спортивная',
 'арбатская',
 'аэропорт',
 'бабушкинская',
 'багратионовская',
 'баррикадная',
 'бауманская',
 'беговая',
 'белорусская',
 'беляево',
 'бибирево',
 'библиотекаим.ленина',
 'новоясеневская',
 'боровицкая',
 'полянка',
 'ботаническийсад',
 'пионерская',
 'солнечная',
 'братиславская',
 'угрешская',
 'коммунарка',
 'бульварадмиралаушакова',
 'бульвардмитриядонского',
 'крымская',
 'щербинка',
 'бунинскаяаллея',
 'варшавская',
 'вднх',
 'владыкино',
 'водныйстадион',
 'войковская',
 'волгоградскийпроспект',
 'волжская',
 'воробьёвыгоры',
 'выхино',
 'москва-сити',
 'динамо',
 'дмитровская',
 'добрынинская',
 'домодедовская',
 'ольховая',
 'дубровка',
 'измайловская',
 'калужская',
 'кантемировская',
 'каховская',
 'каширская',
 'киевская',
 'китай-город',
 'кожуховская',
 'коломенская',
 'комсомольская',
 'коньково',
 'красногвардейск

In [53]:
mtypes

['monolith',
 'brick',
 'block',
 'panel',
 'monolithBrick',
 'stalin',
 'wood',
 'gasSilicateBlock',
 'aerocreteBlock',
 'old',
 'foamConcreteBlock']